In [1]:
import pandas as pd
from lxml import etree
import re
from thefuzz import fuzz

/opt/anaconda3/envs/pouille/lib/python3.9/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#ouverture du fichier XML grâce à etree

xml_file = '../../data/PO_t7/PO_t7.xml'
xml_tree = etree.parse(xml_file)

data = []

#on s'occupe des vedettes: pour l'instant, notre code ne prend pas en compte les vedettes avec + d'une valeur. A voir
#si il faut changer ça. sur le xpath cependant, on dirait qu'il n'y en a pas ?
for article in xml_tree.xpath('//article'):

#on enleve les localisationpa : inutiles pour nous si on ne se focalise que sur la france.
        if article.find('./localisationpa') is not None:
             continue
        vedette_i = article.xpath('./vedette/i')
        vedette_text = vedette_i[0].text if vedette_i else ''

#pareil: il y a un seul localisationde en double, mais il ne le prend pas en compte. A voir si il faut changer ça
        localisationde = article.find('./localisationde')
        localisationde_text = localisationde.text if localisationde is not None else 'none'

#idem. autre version du code qui en prend deux et sépare par ',' existe, à voir si c'est vraiment pertinent.
        localisationca = article.find('./localisationca')
        if localisationca is not None:
            localisationca_text = re.search(r'<localisationca>(.*?)<\/localisationca>', etree.tostring(localisationca, encoding=str)).group(1)
            '''
            cases:
                - 'c<sup>on</sup> de '
                - 'c<sup>on</sup> du '
                - 'c<sup>on</sup> d’'
                - 'c<sup>on</sup> de la '
                - 'c<sup>on</sup> de l’'
                - 'c<sup>on</sup> des '
            '''
            localisationca_text = re.sub(r"c<sup>on</sup> (de |d’|de la |du |de l’|des )", '', localisationca_text)
            
        else:
            localisationca_text = 'none'

#idem. Il n'y en a qu'un avec + d'une localisation -> l'autre étant en suisse. Utile ou pas? à voir.
        localisationco = article.find('./localisationco')
        if localisationco is not None:
            localisationco_text = re.search(r'<localisationco>(.*?)<\/localisationco>', etree.tostring(localisationco, encoding=str)).group(1)            
            '''
            cases:
                - 'c<sup>on</sup> de '
                - 'c<sup>on</sup> du '
                - 'c<sup>on</sup> d’'
                - 'c<sup>on</sup> de la '
                - 'c<sup>on</sup> de l’'
                - 'c<sup>on</sup> des '
                - 'c<sup>nes</sup> de '
                - 'c<sup>nes</sup> d’'
            '''
            localisationco_text = re.sub(r"c<sup>ne(s)?</sup> (de |d’|de la |du |de l’|des )", '', localisationco_text)
            
        else: 
            localisationco_text = 'none'
    
#on insère tout dans un dictionnaire        
        data.append({
            'old-id': article.get('old-id'),
            'vedette': vedette_text,
            'localisationde': localisationde_text,
            'localisationca': localisationca_text,
            'localisationco': localisationco_text
        })

In [3]:
# tous les articles dans un df
df = pd.DataFrame(data)
df.head(25)

,old-id,vedette,localisationde,localisationca,localisationco
0,PO7-00001,Abbaye,Drôme,Die,Die
1,PO7-00002,Abbaye,Isère,Roybon,Marnans
2,PO7-00003,Abbaye,Haute-Savoie,Biot,Saint-Jean-d’Aulph
3,PO7-00005,Abbaye-d’Acey,Jura,Gendrey,Ougney
4,PO7-00006,Abbaye-de-Grandvaux,Jura,Saint-Laurent,Rivière-Devant
5,PO7-00007,Abbaye-de-la-Charité,Haute-Saône,Scey-sur-Saône,Neuvelle-lès-la-Charité
6,PO7-00008,Abbaye-des-Trois-Rois,Doubs,l’Isle-sur-le-Doubs,Mancenans
7,PO7-00009,Abbenans,Doubs,Rougemont,none
8,PO7-00010,Abbevillers,Doubs,Hérimoncourt,none
9,PO7-00011,Aboncourt,Haute-Saône,Combeaufontaine,none


In [4]:
# Ajout du dpt code au df
departements_df = pd.read_csv('departements-region.csv')

# On crée un dictionnaire avec les éléments qui nous intéressent de notre spreadsheet des départements
departements_dict = dict(zip(departements_df['dep_name'], departements_df['num_dep']))

# on ajoute notre colonne avec les numéros des départements qui matchent
df['num_dep'] = df['localisationde'].apply(lambda x: departements_dict.get(x, 'none'))

df.head(5)

,old-id,vedette,localisationde,localisationca,localisationco,num_dep
0,PO7-00001,Abbaye,Drôme,Die,Die,26
1,PO7-00002,Abbaye,Isère,Roybon,Marnans,38
2,PO7-00003,Abbaye,Haute-Savoie,Biot,Saint-Jean-d’Aulph,74
3,PO7-00005,Abbaye-d’Acey,Jura,Gendrey,Ougney,39
4,PO7-00006,Abbaye-de-Grandvaux,Jura,Saint-Laurent,Rivière-Devant,39


In [5]:
#on renomme pour plus de clarté.

df = df.rename(columns={'old-id' : 'article_id', 'localisationca' : 'canton_code', 'localisationco_present' : 'nom_commune', 'num_dep' : 'dpt_code'})
df.head(5)

,article_id,vedette,localisationde,canton_code,localisationco,dpt_code
0,PO7-00001,Abbaye,Drôme,Die,Die,26
1,PO7-00002,Abbaye,Isère,Roybon,Marnans,38
2,PO7-00003,Abbaye,Haute-Savoie,Biot,Saint-Jean-d’Aulph,74
3,PO7-00005,Abbaye-d’Acey,Jura,Gendrey,Ougney,39
4,PO7-00006,Abbaye-de-Grandvaux,Jura,Saint-Laurent,Rivière-Devant,39


In [6]:
#pour encore plus de clarté, nous réorganisons les colonnes 

df = df[['article_id', 'vedette', 'localisationde', 'dpt_code', 'canton_code', 'localisationco']]
df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco
0,PO7-00001,Abbaye,Drôme,26,Die,Die
1,PO7-00002,Abbaye,Isère,38,Roybon,Marnans
2,PO7-00003,Abbaye,Haute-Savoie,74,Biot,Saint-Jean-d’Aulph
3,PO7-00005,Abbaye-d’Acey,Jura,39,Gendrey,Ougney
4,PO7-00006,Abbaye-de-Grandvaux,Jura,39,Saint-Laurent,Rivière-Devant
...,...,...,...,...,...,...
3423,PO7-04280,Zaessingue,Haut-Rhin,68,Landser,none
3424,PO7-04282,Zellenberg,Haut-Rhin,68,Kaysersberg,none
3425,PO7-04285,Zillisheim,Haut-Rhin,68,Mulhouse,none
3426,PO7-04286,Zimmerbach,Haut-Rhin,68,Wintzenheim,none


## Liage des articles de type commune

In [7]:
# Extraction des articles de type commune, que nous pensons à ce stade être des communes.

commune_df = df[df['localisationco'] == 'none']
commune_df['dpt_code'] = commune_df['dpt_code'].apply(lambda x: 'DEP_' + str(x) if x != 'none' else x)

commune_df.head(5)

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/3749711279.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  commune_df['dpt_code'] = commune_df['dpt_code'].apply(lambda x: 'DEP_' + str(x) if x != 'none' else x)


,article_id,vedette,localisationde,dpt_code,canton_code,localisationco
7,PO7-00009,Abbenans,Doubs,DEP_25,Rougemont,none
8,PO7-00010,Abbevillers,Doubs,DEP_25,Hérimoncourt,none
9,PO7-00011,Aboncourt,Haute-Saône,DEP_70,Combeaufontaine,none
10,PO7-00012,Abondance,Haute-Savoie,DEP_74,none,none
11,PO7-00013,Abrets,Isère,DEP_38,Pont-de-Beauvoisin,none


In [8]:
#On lie notre dataframe au référentiel insee avec de l'exact match.

main_insee_commune = pd.read_csv("main_insee_commune.tsv", delimiter='\t')

# on utilise merge pour le liage avec dpt_code et dep_id
commune_merged_df = pd.merge(commune_df, main_insee_commune, left_on='dpt_code', right_on='DEP_id', how='left')

# on sélectionne les lignes où la colonne 'vedette' correspond à la colonne 'NCCENR' de main_insee_commune
matching_rows = commune_merged_df.loc[commune_merged_df['vedette'] == commune_merged_df['NCCENR']]

# on crée une colonne 'insee_code' dans le dataframe commune_df avec les codes correspondants à partir de main_insee_commune
commune_df['insee_code'] = commune_df.apply(lambda row: main_insee_commune.loc[main_insee_commune['NCCENR'] == row['vedette']]['insee_code'].values[0] if len(main_insee_commune.loc[main_insee_commune['NCCENR'] == row['vedette']]['insee_code'].values) > 0 else None, axis=1)
commune_df['insee_code'] = commune_df['insee_code'].fillna('none')

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/1884746733.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  main_insee_commune = pd.read_csv("main_insee_commune.tsv", delimiter='\t')
/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/1884746733.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  commune_df['insee_code'] = commune_df.apply(lambda row: main_insee_commune.loc[main_insee_commune['NCCENR'] == row['vedette']]['insee_code'].values[0] if len(main_insee_commune.loc[main_insee_commune['NCCENR'] == row['vedette']]['insee_code'].values) > 0 else None, axis=1)
/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/1884746733.py:13: SettingWithCop

In [9]:
print(commune_df)

     article_id      vedette localisationde dpt_code         canton_code  \
7     PO7-00009     Abbenans          Doubs   DEP_25           Rougemont   
8     PO7-00010  Abbevillers          Doubs   DEP_25        Hérimoncourt   
9     PO7-00011    Aboncourt    Haute-Saône   DEP_70     Combeaufontaine   
10    PO7-00012    Abondance   Haute-Savoie   DEP_74                none   
11    PO7-00013       Abrets          Isère   DEP_38  Pont-de-Beauvoisin   
...         ...          ...            ...      ...                 ...   
3423  PO7-04280   Zaessingue      Haut-Rhin   DEP_68             Landser   
3424  PO7-04282   Zellenberg      Haut-Rhin   DEP_68         Kaysersberg   
3425  PO7-04285   Zillisheim      Haut-Rhin   DEP_68            Mulhouse   
3426  PO7-04286   Zimmerbach      Haut-Rhin   DEP_68         Wintzenheim   
3427  PO7-04287  Zimmersheim      Haut-Rhin   DEP_68            Habsheim   

     localisationco insee_code  
7              none      25003  
8              none  

In [21]:
#tous nos article_id qui correspondent à un code insee ; on peut donc en conclure que ce sont effectivement des 
#communes.

commune_insee = commune_df[commune_df['insee_code'] != 'none']
print(commune_insee)

#on a donc exactement 2357 matches.

     article_id      vedette localisationde dpt_code          canton_code  \
7     PO7-00009     Abbenans          Doubs   DEP_25            Rougemont   
9     PO7-00011    Aboncourt    Haute-Saône   DEP_70      Combeaufontaine   
10    PO7-00012    Abondance   Haute-Savoie   DEP_74                 none   
11    PO7-00013       Abrets          Isère   DEP_38   Pont-de-Beauvoisin   
12    PO7-00014     Accolans          Doubs   DEP_25  l’Isle-sur-le-Doubs   
...         ...          ...            ...      ...                  ...   
3423  PO7-04280   Zaessingue      Haut-Rhin   DEP_68              Landser   
3424  PO7-04282   Zellenberg      Haut-Rhin   DEP_68          Kaysersberg   
3425  PO7-04285   Zillisheim      Haut-Rhin   DEP_68             Mulhouse   
3426  PO7-04286   Zimmerbach      Haut-Rhin   DEP_68          Wintzenheim   
3427  PO7-04287  Zimmersheim      Haut-Rhin   DEP_68             Habsheim   

     localisationco insee_code  
7              none      25003  
9        

In [22]:
#modification de notre dataframe: ajout d'une colonne method qui nous permet d'identifier la méthode utilisée pour le
#liage

commune_insee['method'] = 'exact'
commune_insee.loc[commune_insee['dpt_code'] == 'none', 'method'] = 'nodpt'

print(commune_insee)

     article_id      vedette localisationde dpt_code          canton_code  \
7     PO7-00009     Abbenans          Doubs   DEP_25            Rougemont   
9     PO7-00011    Aboncourt    Haute-Saône   DEP_70      Combeaufontaine   
10    PO7-00012    Abondance   Haute-Savoie   DEP_74                 none   
11    PO7-00013       Abrets          Isère   DEP_38   Pont-de-Beauvoisin   
12    PO7-00014     Accolans          Doubs   DEP_25  l’Isle-sur-le-Doubs   
...         ...          ...            ...      ...                  ...   
3423  PO7-04280   Zaessingue      Haut-Rhin   DEP_68              Landser   
3424  PO7-04282   Zellenberg      Haut-Rhin   DEP_68          Kaysersberg   
3425  PO7-04285   Zillisheim      Haut-Rhin   DEP_68             Mulhouse   
3426  PO7-04286   Zimmerbach      Haut-Rhin   DEP_68          Wintzenheim   
3427  PO7-04287  Zimmersheim      Haut-Rhin   DEP_68             Habsheim   

     localisationco insee_code method  
7              none      25003  exa

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/692728655.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  commune_insee['method'] = 'exact'


In [23]:
#les articles qui du coup ne seraient apparemment pas des communes.

commune_noinsee = commune_df.loc[commune_df['insee_code'] == 'none']
print(commune_noinsee)

#nous en avons donc 391.

     article_id         vedette localisationde dpt_code  \
8     PO7-00010     Abbevillers          Doubs   DEP_25   
17    PO7-00023    Aiguebelette         Savoie   DEP_73   
22    PO7-00028    Aillevillers    Haute-Saône   DEP_70   
33    PO7-00039           Ajoie           none     none   
35    PO7-00041           Ajoye           none     none   
...         ...             ...            ...      ...   
3378  PO7-04203           Voray    Haute-Saône   DEP_70   
3386  PO7-04212          Vuache           none     none   
3389  PO7-04226           Vully           none     none   
3390  PO7-04227           Vyans    Haute-Saône   DEP_70   
3407  PO7-04253  Wihr-en-Plaine      Haut-Rhin   DEP_68   

                 canton_code localisationco insee_code  
8               Hérimoncourt           none       none  
17        Pont-de-Beauvoisin           none       none  
22    Saint-Loup-sur-Semouse           none       none  
33                      none           none       none  
35    

In [24]:
# Définition d'une fonction pour le fuzzy join

def fuzzy_join(row, distance):
    # Filtre sur les lignes avec les mêmes départements
    main_dep = main_insee_commune[main_insee_commune['DEP_id'] == row['dpt_code']]
    # Calcul de la distance Levenshtein entre la vedette du localisationco_pascommune et les NCCENR de chaque ligne de main_dep
    scores = main_dep['NCCENR'].apply(lambda x: fuzz.token_sort_ratio(x, row['vedette']))
    # Récupération des lignes avec une distance Levenshtein inférieure ou égale à distance
    filtered = main_dep[scores >= distance]
    # Ajout de la colonne insee_code dans le dataframe localisationco_pascommune
    if not filtered.empty:
        insee_code = filtered['insee_code'].values[0]
        if pd.isnull(insee_code):
            return 'none'
        else:
            return insee_code
    else:
        return 'none'
    
# Application de la fonction sur chaque ligne du dataframe localisationco_pascommune
commune_noinsee['insee_code'] = commune_noinsee.apply(fuzzy_join, distance=90, axis=1)

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/354581908.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  commune_noinsee['insee_code'] = commune_noinsee.apply(fuzzy_join, distance=90, axis=1)


In [25]:
print(commune_noinsee)

     article_id         vedette localisationde dpt_code  \
8     PO7-00010     Abbevillers          Doubs   DEP_25   
17    PO7-00023    Aiguebelette         Savoie   DEP_73   
22    PO7-00028    Aillevillers    Haute-Saône   DEP_70   
33    PO7-00039           Ajoie           none     none   
35    PO7-00041           Ajoye           none     none   
...         ...             ...            ...      ...   
3378  PO7-04203           Voray    Haute-Saône   DEP_70   
3386  PO7-04212          Vuache           none     none   
3389  PO7-04226           Vully           none     none   
3390  PO7-04227           Vyans    Haute-Saône   DEP_70   
3407  PO7-04253  Wihr-en-Plaine      Haut-Rhin   DEP_68   

                 canton_code localisationco insee_code  
8               Hérimoncourt           none      25004  
17        Pont-de-Beauvoisin           none       none  
22    Saint-Loup-sur-Semouse           none       none  
33                      none           none       none  
35    

In [26]:
#on veut savoir combien ont été joined par le fuzzy

commune_fuzzy = commune_noinsee[commune_noinsee['insee_code'] != 'none']

# on ajoute une colonne method pour indiquer que le join a été réalisé par fuzzy

commune_fuzzy['method'] = 'fuzzy'

print(commune_fuzzy)

     article_id             vedette localisationde dpt_code     canton_code  \
8     PO7-00010         Abbevillers          Doubs   DEP_25    Hérimoncourt   
144   PO7-00167          Arthemonay          Drôme   DEP_26     Saint-Donat   
237   PO7-00285     Bâtie-des-Fonts          Drôme   DEP_26    Luc-en-Diois   
247   PO7-00296      Baume-d’Hostun          Drôme   DEP_26  Bourg-de-Péage   
265   PO7-00315  Beaumotte-les-Pins    Haute-Saône   DEP_70          Marnay   
...         ...                 ...            ...      ...             ...   
3291  PO7-04102      Villard-d’Héry         Savoie   DEP_73      Montmélian   
3320  PO7-04140   Villeneuve-d’Aval           Jura   DEP_39  Villers-Farlay   
3327  PO7-04147      Villers-Grelot          Doubs   DEP_25         Roulans   
3349  PO7-04170             Vinézac        Ardèche   DEP_07     Largentière   
3367  PO7-04191          Vogelsheim      Haut-Rhin   DEP_68    Neuf-Brisach   

     localisationco insee_code method  
8          

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/536915113.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  commune_fuzzy['method'] = 'fuzzy'


In [27]:
# maintenant, on s'intéresse aux localisationco qui n'ont toujours pas de liage.

commune_fuzzynull = commune_noinsee[commune_noinsee['insee_code'] == 'none']

#idem, on ajoute la méthode pour savoir que ces derniers n'ont été matchés ni avec le exact ni avec le fuzzy.
#on différencie bien sûr ceux n'ayant simplement pas de dpt_code

commune_fuzzynull['method'] = 'nulle'
print(commune_fuzzynull)

     article_id         vedette localisationde dpt_code  \
17    PO7-00023    Aiguebelette         Savoie   DEP_73   
22    PO7-00028    Aillevillers    Haute-Saône   DEP_70   
33    PO7-00039           Ajoie           none     none   
35    PO7-00041           Ajoye           none     none   
37    PO7-00043          Alaise          Doubs   DEP_25   
...         ...             ...            ...      ...   
3378  PO7-04203           Voray    Haute-Saône   DEP_70   
3386  PO7-04212          Vuache           none     none   
3389  PO7-04226           Vully           none     none   
3390  PO7-04227           Vyans    Haute-Saône   DEP_70   
3407  PO7-04253  Wihr-en-Plaine      Haut-Rhin   DEP_68   

                 canton_code localisationco insee_code method  
17        Pont-de-Beauvoisin           none       none  nulle  
22    Saint-Loup-sur-Semouse           none       none  nulle  
33                      none           none       none  nulle  
35                      none       

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/2456345659.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  commune_fuzzynull['method'] = 'nulle'


In [28]:
#il faut maintenant concaténer nos 3 dataframes, en respectant l'ordre original des article_id

commune_final_df = pd.concat([commune_insee, commune_fuzzy, commune_fuzzynull], axis=0)
commune_final_df = commune_final_df.sort_values('method', ascending=True)

print(commune_final_df)

     article_id                vedette localisationde dpt_code  \
7     PO7-00009               Abbenans          Doubs   DEP_25   
2202  PO7-02784            Raedersdorf      Haut-Rhin   DEP_68   
2203  PO7-02785            Raedersheim      Haut-Rhin   DEP_68   
2204  PO7-02786                  Rahon           Jura   DEP_39   
2205  PO7-02787              Raincourt    Haute-Saône   DEP_70   
...         ...                    ...            ...      ...   
1092  PO7-01374  Faymont-et-Vacheresse    Haute-Saône   DEP_70   
1091  PO7-01373           Fay-le-Froid    Haute-Loire   DEP_43   
1082  PO7-01364              Faucogney    Haute-Saône   DEP_70   
1172  PO7-01473               Frickgau           none     none   
3407  PO7-04253         Wihr-en-Plaine      Haut-Rhin   DEP_68   

           canton_code localisationco insee_code method  
7            Rougemont           none      25003  exact  
2202          Ferrette           none      68259  exact  
2203  Soultz-Haut-Rhin           

In [29]:
commune_final_df['origine'] = 'sansloc'

## Liage des communes de localisation (article dans une commune)

Il s’agit des lieux à l’échelle infra-communale. Ces lieux sont localisés dans une commune (`localisationco`)
…

In [30]:
# Liage des articles localisés dans une commune

localisationco_df = df[df['localisationco'] != 'none']
localisationco_df['dpt_code'] = localisationco_df['dpt_code'].apply(lambda x: 'DEP_' + str(x) if x != 'none' else x)
localisationco_df.head(5)

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/811873936.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  localisationco_df['dpt_code'] = localisationco_df['dpt_code'].apply(lambda x: 'DEP_' + str(x) if x != 'none' else x)


,article_id,vedette,localisationde,dpt_code,canton_code,localisationco
0,PO7-00001,Abbaye,Drôme,DEP_26,Die,Die
1,PO7-00002,Abbaye,Isère,DEP_38,Roybon,Marnans
2,PO7-00003,Abbaye,Haute-Savoie,DEP_74,Biot,Saint-Jean-d’Aulph
3,PO7-00005,Abbaye-d’Acey,Jura,DEP_39,Gendrey,Ougney
4,PO7-00006,Abbaye-de-Grandvaux,Jura,DEP_39,Saint-Laurent,Rivière-Devant


In [31]:
#On lie notre dataframe au référentiel insee avec de l'exact match.

main_insee_commune = pd.read_csv("main_insee_commune.tsv", delimiter='\t')

# on utilise merge pour le liage avec dpt_code et dep_id
localisationco_merged_df = pd.merge(localisationco_df, main_insee_commune, left_on='dpt_code', right_on='DEP_id', how='left')

# on sélectionne les lignes où la colonne 'localisationco' correspond à la colonne 'NCCENR' de main_insee_commune
# exact match
matching_rows = localisationco_merged_df.loc[localisationco_merged_df['localisationco'] == localisationco_merged_df['NCCENR']]

# on crée une colonne 'insee_code' dans le dataframe localisationco_df avec les codes correspondants à partir de main_insee_commune
localisationco_df['insee_code'] = localisationco_df.apply(lambda row: main_insee_commune.loc[main_insee_commune['NCCENR'] == row['localisationco']]['insee_code'].values[0] if len(main_insee_commune.loc[main_insee_commune['NCCENR'] == row['localisationco']]['insee_code'].values) > 0 else None, axis=1)
localisationco_df['insee_code'] = localisationco_df['insee_code'].fillna('none')


/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/1939343554.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  main_insee_commune = pd.read_csv("main_insee_commune.tsv", delimiter='\t')
/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/1939343554.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  localisationco_df['insee_code'] = localisationco_df.apply(lambda row: main_insee_commune.loc[main_insee_commune['NCCENR'] == row['localisationco']]['insee_code'].values[0] if len(main_insee_commune.loc[main_insee_commune['NCCENR'] == row['localisationco']]['insee_code'].values) > 0 else None, axis=1)
/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/1939

In [32]:
localisationco_df.head(5)

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,insee_code
0,PO7-00001,Abbaye,Drôme,DEP_26,Die,Die,26113
1,PO7-00002,Abbaye,Isère,DEP_38,Roybon,Marnans,38221
2,PO7-00003,Abbaye,Haute-Savoie,DEP_74,Biot,Saint-Jean-d’Aulph,none
3,PO7-00005,Abbaye-d’Acey,Jura,DEP_39,Gendrey,Ougney,39398
4,PO7-00006,Abbaye-de-Grandvaux,Jura,DEP_39,Saint-Laurent,Rivière-Devant,none


In [33]:
# vérifier les localisationco_communes de localisation liées sur exact match

localisationco_commune = localisationco_df[localisationco_df['insee_code'] != 'none']
localisationco_commune.head(10)

#on a donc exactement 556 matches.

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,insee_code
0,PO7-00001,Abbaye,Drôme,DEP_26,Die,Die,26113
1,PO7-00002,Abbaye,Isère,DEP_38,Roybon,Marnans,38221
3,PO7-00005,Abbaye-d’Acey,Jura,DEP_39,Gendrey,Ougney,39398
5,PO7-00007,Abbaye-de-la-Charité,Haute-Saône,DEP_70,Scey-sur-Saône,Neuvelle-lès-la-Charité,70384
6,PO7-00008,Abbaye-des-Trois-Rois,Doubs,DEP_25,l’Isle-sur-le-Doubs,Mancenans,25365
20,PO7-00026,Aiguebonne,Drôme,DEP_26,Crest,Allex,26006
42,PO7-00048,Alberges,Savoie,DEP_73,Chambéry,Saint-Cassin,73228
58,PO7-00067,Allier,Loire,DEP_42,Bourg-Argental,Bourg-Argental,42023
63,PO7-00073,Alschwiller,Haut-Rhin,DEP_68,Soultz-Haut-Rhin,Soultz-Haut-Rhin,68315
64,PO7-00074,Alspach,Haut-Rhin,DEP_68,Kaysersberg,Kaysersberg,68162


In [34]:
print(localisationco_commune)

     article_id                vedette localisationde dpt_code  \
0     PO7-00001                 Abbaye          Drôme   DEP_26   
1     PO7-00002                 Abbaye          Isère   DEP_38   
3     PO7-00005          Abbaye-d’Acey           Jura   DEP_39   
5     PO7-00007   Abbaye-de-la-Charité    Haute-Saône   DEP_70   
6     PO7-00008  Abbaye-des-Trois-Rois          Doubs   DEP_25   
...         ...                    ...            ...      ...   
3401  PO7-04245               Weinbach      Haut-Rhin   DEP_68   
3402  PO7-04247             Weisskirch      Haut-Rhin   DEP_68   
3411  PO7-04259            Wintzfelden      Haut-Rhin   DEP_68   
3416  PO7-04268             Wolfenheim      Haut-Rhin   DEP_68   
3421  PO7-04276                    Yon            Ain   DEP_01   

              canton_code           localisationco insee_code  
0                     Die                      Die      26113  
1                  Roybon                  Marnans      38221  
3              

In [35]:
#modification de notre dataframe: ajout d'une colonne method qui nous permet d'identifier la méthode utilisée pour le
#liage

localisationco_commune['method'] = 'exact'
# marquer les liages exécutés sans possibilité de désambiguiser le dpt (pour validation manuelle a posteriori)
localisationco_commune.loc[localisationco_commune['dpt_code'] == 'none', 'method'] = 'nodpt'

print(localisationco_commune)

     article_id                vedette localisationde dpt_code  \
0     PO7-00001                 Abbaye          Drôme   DEP_26   
1     PO7-00002                 Abbaye          Isère   DEP_38   
3     PO7-00005          Abbaye-d’Acey           Jura   DEP_39   
5     PO7-00007   Abbaye-de-la-Charité    Haute-Saône   DEP_70   
6     PO7-00008  Abbaye-des-Trois-Rois          Doubs   DEP_25   
...         ...                    ...            ...      ...   
3401  PO7-04245               Weinbach      Haut-Rhin   DEP_68   
3402  PO7-04247             Weisskirch      Haut-Rhin   DEP_68   
3411  PO7-04259            Wintzfelden      Haut-Rhin   DEP_68   
3416  PO7-04268             Wolfenheim      Haut-Rhin   DEP_68   
3421  PO7-04276                    Yon            Ain   DEP_01   

              canton_code           localisationco insee_code method  
0                     Die                      Die      26113  exact  
1                  Roybon                  Marnans      38221  ex

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/1184485917.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  localisationco_commune['method'] = 'exact'


In [36]:
#les articles qui du coup n'ont pas de liage. on fera le fuzzy pour eux.

localisationco_pascommune = localisationco_df.loc[localisationco_df['insee_code'] == 'none']
print(localisationco_pascommune)

#nous en avons donc 124.

     article_id              vedette localisationde dpt_code      canton_code  \
2     PO7-00003               Abbaye   Haute-Savoie   DEP_74             Biot   
4     PO7-00006  Abbaye-de-Grandvaux           Jura   DEP_39    Saint-Laurent   
13    PO7-00015               Achard          Isère   DEP_38  Villard-de-Lans   
47    PO7-00054              Alençon          Drôme   DEP_26        Dieulefit   
92    PO7-00106             Annegray    Haute-Saône   DEP_70        Faucogney   
...         ...                  ...            ...      ...              ...   
3242  PO7-04035             Vermelle          Isère   DEP_38         Bourgoin   
3243  PO7-04037            Vernaison          Drôme   DEP_26   Bourg-de-Péage   
3264  PO7-04062               Vetraz   Haute-Savoie   DEP_74        Annemasse   
3310  PO7-04128                Ville          Isère   DEP_38   la Verpillière   
3316  PO7-04134           Villeneuve          Isère   DEP_38           Domène   

              localisationc

In [37]:
# Définition d'une fonction pour le fuzzy join

def fuzzy_join(row, distance):
    # Filtre sur les lignes avec les mêmes départements
    main_dep = main_insee_commune[main_insee_commune['DEP_id'] == row['dpt_code']]
    # Calcul de la distance Levenshtein entre la vedette du localisationco_pascommune et les NCCENR de chaque ligne de main_dep
    scores = main_dep['NCCENR'].apply(lambda x: fuzz.token_sort_ratio(x, row['vedette']))
    # Récupération des lignes avec une distance Levenshtein inférieure ou égale à distance
    filtered = main_dep[scores >= distance]
    # Ajout de la colonne insee_code dans le dataframe localisationco_pascommune
    if not filtered.empty:
        insee_code = filtered['insee_code'].values[0]
        if pd.isnull(insee_code):
            return 'none'
        else:
            return insee_code
    else:
        return 'none'
    
# Application de la fonction sur chaque ligne du dataframe localisationco_pascommune
localisationco_pascommune['insee_code'] = localisationco_pascommune.apply(fuzzy_join, distance=90, axis=1)

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/3614077139.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  localisationco_pascommune['insee_code'] = localisationco_pascommune.apply(fuzzy_join, distance=90, axis=1)


In [38]:
print(localisationco_pascommune)

     article_id              vedette localisationde dpt_code      canton_code  \
2     PO7-00003               Abbaye   Haute-Savoie   DEP_74             Biot   
4     PO7-00006  Abbaye-de-Grandvaux           Jura   DEP_39    Saint-Laurent   
13    PO7-00015               Achard          Isère   DEP_38  Villard-de-Lans   
47    PO7-00054              Alençon          Drôme   DEP_26        Dieulefit   
92    PO7-00106             Annegray    Haute-Saône   DEP_70        Faucogney   
...         ...                  ...            ...      ...              ...   
3242  PO7-04035             Vermelle          Isère   DEP_38         Bourgoin   
3243  PO7-04037            Vernaison          Drôme   DEP_26   Bourg-de-Péage   
3264  PO7-04062               Vetraz   Haute-Savoie   DEP_74        Annemasse   
3310  PO7-04128                Ville          Isère   DEP_38   la Verpillière   
3316  PO7-04134           Villeneuve          Isère   DEP_38           Domène   

              localisationc

In [39]:
#on veut savoir combien ont été joined par le fuzzy_localisationco

fuzzy_localisationco = localisationco_pascommune[localisationco_pascommune['insee_code'] != 'none']

# on ajoute une colonne method pour indiquer que le join a été réalisé par fuzzy_localisationco

fuzzy_localisationco['method'] = 'fuzzy'


print(fuzzy_localisationco)

     article_id       vedette localisationde dpt_code          canton_code  \
124   PO7-00143      Aresches           Jura   DEP_39               Salins   
382   PO7-00476     Bonnevaux           Gard   DEP_30             Génolhac   
913   PO7-01136          Cusy   Haute-Savoie   DEP_74             Douvaine   
1181  PO7-01486        Funans          Doubs   DEP_25  l’Isle-sur-le-Doubs   
2361  PO7-02972  Saint-Andéol          Drôme   DEP_26             Marsanne   

            localisationco insee_code method  
124      Moutaine-Aresches      39586  fuzzy  
382   Bonnevaux-et-Hiverne      30044  fuzzy  
913                  Chens      74097  fuzzy  
1181   l’Isle-sur-le-Doubs      25262  fuzzy  
2361      la Bâtie-Rolland      26291  fuzzy  


/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/2428670033.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuzzy_localisationco['method'] = 'fuzzy'


In [40]:
# maintenant, on s'intéresse aux localisationco qui n'ont toujours pas de liage.

fuzzynull_localisationco = localisationco_pascommune[localisationco_pascommune['insee_code'] == 'none']

#idem, on ajoute la méthode pour savoir que ces derniers n'ont été matchés ni avec le exact ni avec le fuzzy

fuzzynull_localisationco['method'] = 'nulle'
print(fuzzynull_localisationco)

     article_id              vedette localisationde dpt_code      canton_code  \
2     PO7-00003               Abbaye   Haute-Savoie   DEP_74             Biot   
4     PO7-00006  Abbaye-de-Grandvaux           Jura   DEP_39    Saint-Laurent   
13    PO7-00015               Achard          Isère   DEP_38  Villard-de-Lans   
47    PO7-00054              Alençon          Drôme   DEP_26        Dieulefit   
92    PO7-00106             Annegray    Haute-Saône   DEP_70        Faucogney   
...         ...                  ...            ...      ...              ...   
3242  PO7-04035             Vermelle          Isère   DEP_38         Bourgoin   
3243  PO7-04037            Vernaison          Drôme   DEP_26   Bourg-de-Péage   
3264  PO7-04062               Vetraz   Haute-Savoie   DEP_74        Annemasse   
3310  PO7-04128                Ville          Isère   DEP_38   la Verpillière   
3316  PO7-04134           Villeneuve          Isère   DEP_38           Domène   

              localisationc

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_5983/917059299.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuzzynull_localisationco['method'] = 'nulle'


In [41]:
#il faut maintenant concaténer nos 3 dataframes, en respectant l'ordre original des article_id

localisationco_final_df = pd.concat([localisationco_commune, fuzzy_localisationco, fuzzynull_localisationco], axis=0)
localisationco_final_df = localisationco_final_df.sort_values('method', ascending=True)

print(localisationco_final_df)

     article_id                 vedette localisationde dpt_code  \
0     PO7-00001                  Abbaye          Drôme   DEP_26   
2466  PO7-03100            Sainte-Croix          Loire   DEP_42   
2464  PO7-03098            Sainte-Croix        Ardèche   DEP_07   
2462  PO7-03095        Sainte-Catherine   Haute-Savoie   DEP_74   
2457  PO7-03089            Sainte-Affre      Haut-Rhin   DEP_68   
...         ...                     ...            ...      ...   
1100  PO7-01383               Fenestroz         Savoie   DEP_73   
1075  PO7-01354                   Eyzin          Isère   DEP_38   
1040  PO7-01311                   Ésery   Haute-Savoie   DEP_74   
1939  PO7-02447  Notre-Dame-de-la-Gorge   Haute-Savoie   DEP_74   
3316  PO7-04134              Villeneuve          Isère   DEP_38   

                  canton_code         localisationco insee_code method  
0                         Die                    Die      26113  exact  
2466             Rive-de-Gier                Pave

In [42]:
localisationco_final_df['origine'] = 'avecloc'

## Fusion et ordonnancement des liages

In [43]:
# Créer une colonne de correspondance pour les deux dataframes
commune_final_df['corresponding_value'] = commune_final_df['vedette']
localisationco_final_df['corresponding_value'] = localisationco_final_df['localisationco']

# Concaténer les deux dataframes
merged_df = pd.concat([commune_final_df, localisationco_final_df], sort=False)

# Trier les valeurs par ordre de la colonne "corresponding_value" puis par ordre de la colonne "article_id"
merged_df.sort_values(by=['corresponding_value', 'article_id'], inplace=True)

# Supprimer la colonne "corresponding_value"
merged_df.drop(columns=['corresponding_value'], inplace=True)

print(merged_df)

     article_id          vedette localisationde dpt_code      canton_code  \
1511  PO7-01925        Lieu-Dieu          Doubs   DEP_25       Boussières   
7     PO7-00009         Abbenans          Doubs   DEP_25        Rougemont   
8     PO7-00010      Abbevillers          Doubs   DEP_25     Hérimoncourt   
9     PO7-00011        Aboncourt    Haute-Saône   DEP_70  Combeaufontaine   
10    PO7-00012        Abondance   Haute-Savoie   DEP_74             none   
...         ...              ...            ...      ...              ...   
1060  PO7-01339             Étuz    Haute-Saône   DEP_70           Marnay   
1062  PO7-01341            Évans           Jura   DEP_39        Dampierre   
1063  PO7-01342  Évian-les-Bains   Haute-Savoie   DEP_74             none   
1064  PO7-01343           Évires   Haute-Savoie   DEP_74          Thorens   
1065  PO7-01344          Évosges            Ain   DEP_01    Saint-Rambert   

      localisationco insee_code method  origine  
1511  Abbans-Dessous     

In [47]:
merged_df = pd.merge(merged_df, main_insee_commune[['NCCENR', 'insee_code']], on='insee_code', how='left')
merged_df['NCCENR'] = merged_df['NCCENR'].fillna('none')

In [48]:
print(merged_df)

     article_id          vedette localisationde dpt_code      canton_code  \
0     PO7-01925        Lieu-Dieu          Doubs   DEP_25       Boussières   
1     PO7-00009         Abbenans          Doubs   DEP_25        Rougemont   
2     PO7-00010      Abbevillers          Doubs   DEP_25     Hérimoncourt   
3     PO7-00011        Aboncourt    Haute-Saône   DEP_70  Combeaufontaine   
4     PO7-00012        Abondance   Haute-Savoie   DEP_74             none   
...         ...              ...            ...      ...              ...   
3423  PO7-01339             Étuz    Haute-Saône   DEP_70           Marnay   
3424  PO7-01341            Évans           Jura   DEP_39        Dampierre   
3425  PO7-01342  Évian-les-Bains   Haute-Savoie   DEP_74             none   
3426  PO7-01343           Évires   Haute-Savoie   DEP_74          Thorens   
3427  PO7-01344          Évosges            Ain   DEP_01    Saint-Rambert   

      localisationco insee_code method  origine           NCCENR  
0     Ab

In [49]:
method_counts = merged_df['method'].value_counts()
print(method_counts)

exact    2845
nulle     398
fuzzy     117
nodpt      68
Name: method, dtype: int64


In [50]:
origine_counts = merged_df['origine'].value_counts()
print(origine_counts)

sansloc    2748
avecloc     680
Name: origine, dtype: int64


In [51]:
# Sauvegarder le dataframe fusionné
merged_df.to_csv('../../utils/liage_po7.csv', index=False)